In [1]:
import numpy as np
from scipy import stats
from scipy.stats import skew
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.externals import joblib   # 함수는 dump 시켜도 안됨
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [4]:
from my_transformer import change_to_str, divide_columns, feature_selection, simple_imputer, one_hot_encoding, concat, rf_imputer, fill_columns
from my_transformer import rmsle_scorer, neg_rmsle_scorer, rmsle

In [5]:
def preparation(data) :
    
#     data.drop(['SalePrice', 'Id'], axis=1, inplace=True)
    data.drop(['Id'], axis=1, inplace=True)
    
    astype_str = change_to_str('MSSubClass')
    data = astype_str.fit_transform(data)
    
    num_columns, cat_columns = divide_columns(data)
    
    num_null_0 = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                  'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']
    num_null_mean = 'LotFrontage'
    num_null_rf = 'GarageYrBlt'
    replace_null = ['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC',
                   'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageFinish', 'GarageCond']
    
    for column in replace_null :
        data.loc[data[column].isnull(), column] = 'NA'
    
    pipeline_cat = make_pipeline(
        feature_selection(cat_columns),
        simple_imputer('most_frequent'),
        one_hot_encoding(cat_columns)
    )
    
    X_cat = pipeline_cat.fit_transform(data)
    X_num = data[num_columns]
    X = concat(X_num, X_cat)
    
    isnull_sum = X.isnull().sum()
    not_null = list(isnull_sum[isnull_sum == 0].index)
    null_columns = list(isnull_sum[isnull_sum > 0])
    
    for column in num_columns :
        if column == num_null_rf :
            X = rf_imputer(X, column, not_null)

        elif column == num_null_mean :
            X = fill_columns(X, column, 'mean')

        else :
            X = fill_columns(X, column, 'constant', 0)
        
    isnull_sum = X.isnull().sum()
    print(isnull_sum[isnull_sum > 0].sort_values(ascending=False))
    
    skew_features = X[num_columns].apply(lambda x : skew(x))
    skew_features_top = skew_features[skew_features > 1]
    X[skew_features_top.index] = np.log1p(X[skew_features_top.index])
            
    return X

In [6]:
def data_set() :
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    
    data = pd.concat([train, test], axis=0)
    
    X = preparation(data)
    X_test = X.iloc[1460:, :]
    X = X.iloc[:1460, :]
    
    y = train['SalePrice']
    y = np.log1p(y)
    
    return X, X_test, y

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [8]:
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [9]:
data = pd.concat([train, test], axis=0)
data.shape

(2919, 81)

In [10]:
X = preparation(data)

Series([], dtype: int64)


In [11]:
X.shape

(2919, 316)

In [12]:
X_test = X.iloc[1460:, :]
X = X.iloc[:1460, :]

In [13]:
print(X.shape)
print(X_test.shape)

(1460, 316)
(1459, 316)


In [14]:
corr_matrix = X.corr()
corr_matrix['SalePrice'].sort_values(ascending=False)

SalePrice                1.000000
GarageCond_TA            0.621334
GarageCars               0.554501
GarageArea               0.528420
PavedDrive_Y             0.365585
OverallQual              0.341372
CentralAir_Y             0.322836
GarageYrBlt              0.297975
YearBuilt                0.268198
Fireplaces               0.248425
GrLivArea                0.215779
1stFlrSF                 0.209862
YearRemodAdd             0.195974
MasVnrArea               0.190389
ExterQual_Gd             0.189039
MSZoning_RL              0.188719
GarageFinish_Fin         0.186316
BsmtFinSF1               0.181266
GarageFinish_RFn         0.178650
BldgType_1Fam            0.175332
WoodDeckSF               0.166377
TotalBsmtSF              0.166152
MSSubClass_60            0.165335
LotShape_IR1             0.162507
LotArea                  0.161402
LotFrontage              0.158675
FullBath                 0.149678
HeatingQC_Ex             0.149148
FireplaceQu_Gd           0.149035
BsmtCond_TA   

In [15]:
feature_selection = abs(corr_matrix['SalePrice']).sort_values(ascending=False).head(20)
feature_selection

SalePrice              1.000000
GarageFinish_NA        0.707311
GarageCond_NA          0.707311
GarageCond_TA          0.621334
GarageCars             0.554501
GarageArea             0.528420
PavedDrive_N           0.398102
PavedDrive_Y           0.365585
OverallQual            0.341372
CentralAir_N           0.322836
CentralAir_Y           0.322836
GarageYrBlt            0.297975
FireplaceQu_NA         0.270267
YearBuilt              0.268198
Fireplaces             0.248425
GrLivArea              0.215779
MSZoning_C (all)       0.212862
1stFlrSF               0.209862
Neighborhood_IDOTRR    0.203124
YearRemodAdd           0.195974
Name: SalePrice, dtype: float64

In [16]:
X = X[feature_selection.index]
X_test = X_test[feature_selection.index]

In [17]:
X.shape

(1460, 20)

In [18]:
X.drop('SalePrice', axis=1, inplace=True)
X_test.drop('SalePrice', axis=1, inplace=True)
y = train['SalePrice']
y = np.log1p(y)

In [19]:
knn = KNeighborsRegressor(n_neighbors=5)
linear = LinearRegression()
sgd = SGDRegressor(max_iter=None, eta0=0.01, penalty='l2', random_state=30)
ridge = Ridge(alpha=1, random_state=30)
lasso = Lasso(alpha=1, random_state=30)
elastic = ElasticNet(alpha=1, l1_ratio=0.5, random_state=30)
dt = DecisionTreeRegressor(max_depth=None, random_state=30)
rf = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=30, n_jobs=-1)
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=30)
svm = SVR(C=1, kernel='rbf', gamma='auto_deprecated')
mlp = MLPRegressor(hidden_layer_sizes=[100], solver='adam', activation='relu', alpha=0.0001, max_iter=200, random_state=30)
xgb = XGBRegressor(random_state=30)

In [20]:
models = {
    'knn' : knn,
    'linear' : linear,
    'sgd' : sgd,
    'ridge' : ridge,
    'lasso' : lasso,
    'elastic' : elastic,
    'dt' : dt,
    'rf' : rf,
    'gb' : gb,
    'xgb' : xgb,
    'svm' : svm,
    'mlp' : mlp
}

In [21]:
scores = dict()

for key, model in models.items() : 
    score = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error').mean()   
    scores[key] = round(np.sqrt(-score), 3)
    
scores

[10:27:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:27:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:27:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:27:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:27:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'knn': 0.445,
 'linear': 0.403,
 'sgd': 1.1187249470427262e+16,
 'ridge': 0.403,
 'lasso': 0.4,
 'elastic': 0.4,
 'dt': 0.602,
 'rf': 0.419,
 'gb': 0.418,
 'xgb': 0.41,
 'svm': 0.425,
 'mlp': 3.103}